### Agents
Agents in essence are LLMs that accomplish a specific task. They can be supplemented with tools, structured output, and handoff to other agents.

In [1]:
from dotenv import load_dotenv
import os

# loads the .env file (if you have a global environment variable, you can skip this)
load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables")

In [2]:
from agents import Agent, Runner

agent = Agent(
    name="Basic Agent",
    instructions="You are a helpful assistant. Respond on in all caps.",
    model="gpt-4o-mini"
)

result = await Runner.run(agent, "Hello! How are you?")
result.final_output

Error getting response: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************reEA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}. (request_id: req_4d6f5879494da0468c2b39856486df59)


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************reEA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

[non-fatal] Tracing client error 401: {
  "error": {
    "message": "Incorrect API key provided: sk-proj-********************************************************************************************************************************************************reEA. You can find your API key at https://platform.openai.com/account/api-keys.",
    "type": "invalid_request_error",
    "param": null,
    "code": "invalid_api_key"
  }
}


In [ ]:
joke_agent = Agent(
    name="Joke Agent",
    instructions="You are a joke teller. You are given a topic and you need to tell a joke about it.",
)

topic = "Boogers"
result = await Runner.run(joke_agent, topic)
result.final_output

In [ ]:
language_agent = Agent(
    name="Language Agent",
    instructions="You are a language expert. You are given a joke and you need to rewrite it in a different language.",
)

joke_result = await Runner.run(joke_agent, topic)
translated_result = await Runner.run(language_agent, f"Translate this joke to Spanish: {joke_result.final_output}")
print(f"Original joke:\n{joke_result.final_output}\n")
print(f"Translated joke:\n{translated_result.final_output}")

### Structured Outputs
Structured outputs are a way to format the output of an LLM in a structured manner. This can be useful for tasks that require specific formatting or data extraction.

In [ ]:
from pydantic import BaseModel
from agents import Agent

class Recipe(BaseModel):
    title: str
    ingredients: list[str]
    cooking_time: int # in minutes
    servings: int

recipe_agent = Agent(
    name="Recipe Agent", 
    instructions=("You are an agent for creating recipes. You will be given the name of a food and your job"
                  " is to output that as an actual detailed recipe. The cooking time should be in minutes."),
    output_type=Recipe
)

response = await Runner.run(recipe_agent, "Italian Sasuage with Spaghetti")
recipe = response.final_output

### Tool Calling
Tool calling is a way to extend the capabilities of an LLM by allowing it to call external tools or APIs. This can be useful for tasks that require access to external data or services.

In [ ]:
from agents import Agent, function_tool

@function_tool
def get_weather(city: str) -> str:
    print(f"Getting weather for {city}")
    return "sunny"

@function_tool
def get_temperature(city: str) -> str:
    print(f"Getting temperature for {city}")
    return "70 degrees"

agent = Agent(
    name="Weather Agent",
    instructions="You are the local weather agent. You are given a city and you need to tell the weather and temperature. For any unrelated queries, say I cant help with that.",
    tools=[get_weather, get_temperature]
)

result = await Runner.run(agent, "Dallas")
result.final_output

In [ ]:
from agents import WebSearchTool

news_agent = Agent(
    name="News Reporter",
    instructions="You are a news reporter. Your job is to find recent news articles on the internet about US politics.",
    tools=[WebSearchTool()]
)

result = await Runner.run(news_agent, "find news")
print(result.final_output)